In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Define surveys

In [ ]:
dragons1    = {'file':'dragons',   'name':'DRAGONS 800-1030',   'fmin':800e6,  'fmax':1030e6, 'df':1e6}
dragons2    = {'file':'dragons',   'name':'DRAGONS 700-1030',   'fmin':700e6,  'fmax':1030e6, 'df':1e6}
dragons_all = {'file':'dragons',   'name':'DRAGONS 350-1030',   'fmin':350e6,  'fmax':1030e6, 'df':1e6}
chime_full = {'file':'chime_full','name':'CHIME ',    'fmin':400e6,  'fmax':800e6,  'df':0.4e6}
chime_low  = {'file':'chime_low', 'name':'CHIME low', 'fmin':400e6,  'fmax':440e6,  'df':0.4e6}
chime_high = {'file':'chime_high','name':'CHIME high','fmin':586e6,  'fmax':729e6,  'df':0.4e6}
chime_new = {'file':'chime','name':'CHIME 400-729',    'fmin':400e6,  'fmax':729e6,  'df':1e6}
gmims_hbn  = {'file':'gmims_hbn', 'name':'GMIMS HBN', 'fmin':1280e6, 'fmax':1750e6, 'df':1e6}
gmims_lbs  = {'file':'gmims_lbs', 'name':'GMIMS LBS', 'fmin':300e6,  'fmax':480e6,  'df':0.5e6}
gmims_full = {'file':'gmims_full', 'name':' ', 'fmin':300e6,  'fmax':1800e6,  'df':0.5e6}
chime_drag = {'file':'dragons_chime',   'name':'CHIME+DRAGONS 512-729',   'fmin':400e6,  'fmax':512e6, 'df':1e6}
#staps      = {'file':, 'name':,'fmin': , 'fmax': ,  'df':}
#pegasus    = {'file':, 'name':, fmin': , 'fmax': ,  'df':}

## Pick the surveys to plot, and FD range and sampling

In [ ]:
phi_max    = 150 # range of FD values to plot for RMSF (+/-)
dphi       = 0.5 # FD sampling
survey_all = [dragons_all,chime_new,chime_drag] # which surveys to plot (1 or more)
colors     = ['C0','C1','C2'] # colors to use for plotting each survey 
combine    = False # Set True if want to combine frequency coverage from chosen surveys
file_out   = 'chime_dragons_f2f_2024.jpg' # name of the file to save

## Some functions

In [ ]:
c = 2.998e8
freq = np.asarray(np.linspace(200,2000,2001))
lbd2 = (c/(freq*1e6))**2

def get_survey_params(survey):
    
    phi = []
    rmsf= []
    with open(survey['file']+'.txt', 'r') as file:
        for line in file:
            phi.append(float(line.split()[0]))
            rmsf.append(float(line.split()[1]))

    with open(survey['file']+'_params.txt', 'r') as file:
        for line in file:
            if line.split()[0] == 'phi_broad':
                phi_broad = float(line.split()[1])
    
    return phi, rmsf, phi_broad

def get_combo_survey_params():
    
    phi = []
    rmsf= []
    with open('combo.txt', 'r') as file:
        for line in file:
            phi.append(float(line.split()[0]))
            rmsf.append(float(line.split()[1]))

    with open('combo_params.txt', 'r') as file:
        for line in file:
            if line.split()[0] == 'phi_broad':
                phi_broad = float(line.split()[1])
            if line.split()[0] == 'FWHM':
                res = float(line.split()[1])
    
    return phi, rmsf, phi_broad, res

def make_the_plot(survey_all,colors,freqs_all,phi_broad_all,phi_all,rmsf_all,
                  phi_combo=None, rmsf_combo=None, phi_broad_combo=None, do_combo=False,inc_legend=True,
                  fs=24,save_plot=True,file_out='test.jpg',*args,**kwargs):
    
    fig, ax = plt.subplots(figsize=(25,8))

    # Make the coverage plot:

    ax.plot(lbd2,freq,color='black',zorder=100)
    for j in range(0,len(freqs_all)):
        freqs = freqs_all[j]
        y = np.arange(freqs[0],freqs[1],0.1*1e6)
        x = (c/y)**2
        x1 = (np.arange(0,c/freqs[0],0.001))**2
        x2 = (np.arange(0,c/freqs[1],0.001))**2
        y1 = np.repeat(freqs[0],len(x1))
        y2 = np.repeat(freqs[1],len(x2))

        ax.plot(x1,y1/1e6,color=colors[j])
        ax.plot(x2,y2/1e6,color=colors[j])
        ax.plot([x1[-1],x1[-1]],[0,y1[0]/1e6],color=colors[j],label=survey_all[j]['name'])
        ax.plot([x2[-1],x2[-1]],[0,y2[0]/1e6],color=colors[j])

        ax.fill_between(x,y/1e6,color=colors[j],alpha=0.4)
        ax.fill_between(x2,y2/1e6,y1[0:len(x2)]/1e6,color=colors[j],alpha=0.4)

    if inc_legend:
        ax.legend(fontsize=fs,loc=(0.08,0.7))
    ax.set_xticks(np.arange(0.1,1.1,0.1))
    ax.tick_params(labelsize=fs,axis='both')
    ax.set_xlim(0,1)
    ax.set_ylim(200,1800)
    ax.set_xlabel('$\lambda^2$ (m$^2$)',fontsize=fs)
    ax.set_ylabel(r'$\nu$ (MHz)',fontsize=fs)

    # Make the RMSF plot:

    insetPosition = [0.395, 0.4, 0.49, 0.45]
    ax2 = fig.add_axes(insetPosition)
    for j in range(0,len(freqs_all)):
        ax2.plot(phi_all[j], rmsf_all[j],color=colors[j],linewidth=2)
        ax2.plot([-phi_broad_all[j]/2,-phi_broad_all[j]/2],[0,1.1],color=colors[j],
                 linestyle='dashed',linewidth = 2)
        ax2.plot([ phi_broad_all[j]/2, phi_broad_all[j]/2],[0,1.1],color=colors[j],
                 linestyle='dashed',linewidth = 2)
    if do_combo:
        ax2.plot(phi_combo, rmsf_combo,color='k',linewidth=2)
        ax2.plot([-phi_broad_combo/2,-phi_broad_combo/2],[0,1.1],color='k',
                 linestyle='dashed',linewidth = 2)
        ax2.plot([ phi_broad_combo/2, phi_broad_combo/2],[0,1.1],color='k',
                 linestyle='dashed',linewidth = 2)
    ax2.set_xticks(np.arange(-300,400,100))
    ax2.tick_params(labelsize=fs,axis='both')
    ax2.set_xlim(-phi_max,phi_max)
    ax2.set_ylim(0,1.1)
    ax2.set_xlabel('Faraday Depth (rad m$^{-2}$)',fontsize=fs)
    ax2.set_ylabel('RMSF',fontsize=fs)

    plt.savefig(file_out)

    return

## Calculate RMSFs

In [ ]:
freqs_all     = []
phi_broad_all = []
phi_all       = []
rmsf_all      = []

for survey in survey_all:
    
    fmin = survey['fmin']
    fmax = survey['fmax']
    df = survey['df']
    RMSFout = survey['file']

    cmd = 'python3 calculate_RMSF_AO.py -m '+str(phi_max)+' -d '+str(dphi)+' -f '+str(fmin)+' '+str(fmax)+' '+str(df)+' -o '+RMSFout
    print(cmd)
    !{cmd}
    
    freqs_all.append([survey['fmin'],survey['fmax']])
    phi, rmsf, phi_broad = get_survey_params(survey)
    phi_broad_all.append(phi_broad)
    phi_all.append(phi)
    rmsf_all.append(rmsf)
    
if combine:
    fcombine = []
    for survey in survey_all:
        fcombine.extend(list(np.arange(survey['fmin'],survey['fmax']+survey['df'],survey['df'])))
    with open('combo_freq.txt', 'w') as filehandle:
        for i in range(0,len(fcombine)):
            filehandle.write(f"{fcombine[i]}\n")
    cmd = 'python3 calculate_RMSF_AO.py combo_freq.txt -m '+str(phi_max)+' -d '+str(dphi)+' -o combo'
    !{cmd}
    phi_combo, rmsf_combo, phi_broad_combo = get_combo_survey_params()
    
cmd = 'rm *.txt'
!{cmd}

## Make the plot

In [ ]:
make_the_plot(survey_all,colors,freqs_all,phi_broad_all,phi_all,rmsf_all,fs=24,
              phi_combo=None, rmsf_combo=None, phi_broad_combo=None,inc_legend=False,
              do_combo=combine,file_out=file_out)

In [ ]:
2*np.sqrt(3)/((3e8/350e6)**2 - (3e8/1800e6)**2)

# Version that includes RFI masking

In [ ]:
c = 2.998e8
freq = np.asarray(np.linspace(200,2000,2001))
lbd2 = (c/(freq*1e6))**2

def make_the_plot_rfi(survey,color,phi_broad,phi,rmsf,
                  fs=24,save_plot=True,file_out='test.jpg',*args,**kwargs):
    
    fig, ax = plt.subplots(figsize=(25,8))

    # Make the coverage plot:

    ax.plot(lbd2,freq,color='black',zorder=100)
    
    freqs_all = survey['frange']
    
    for j in range(0,len(freqs_all)):
        freqs = freqs_all[j]
        y = np.arange(freqs[0],freqs[1],0.1*1e6)
        x = (c/y)**2
        x1 = (np.arange(0,c/freqs[0],0.001))**2
        x2 = (np.arange(0,c/freqs[1],0.001))**2
        y1 = np.repeat(freqs[0],len(x1))
        y2 = np.repeat(freqs[1],len(x2))

        ax.plot(x1,y1/1e6,color=color)
        ax.plot(x2,y2/1e6,color=color)
        ax.plot([x1[-1],x1[-1]],[0,y1[0]/1e6],color=color,label=survey['name'])
        ax.plot([x2[-1],x2[-1]],[0,y2[0]/1e6],color=color)

        ax.fill_between(x,y/1e6,color=color,alpha=0.4)
        ax.fill_between(x2,y2/1e6,y1[0:len(x2)]/1e6,color=color,alpha=0.4)

    #ax.legend(fontsize=fs,loc=(0.08,0.7))
    ax.set_xticks(np.arange(0.1,1.1,0.1))
    ax.tick_params(labelsize=fs,axis='both')
    #ax.set_xlim(0,1)
    ax.set_xlim(0.0,1.0)
    ax.set_ylim(250,1900)
    ax.set_xlabel('$\lambda^2$ (m$^2$)',fontsize=fs)
    ax.set_ylabel(r'$\nu$ (MHz)',fontsize=fs)

    # Make the RMSF plot:

    #insetPosition = [0.395, 0.4, 0.49, 0.45]
    insetPosition = [0.42, 0.4, 0.465, 0.45]
    ax2 = fig.add_axes(insetPosition)
    #ax2.plot(phi, rmsf,color=color,linewidth=2)
    ax2.plot(phi, rmsf,color='k',linewidth=2)
    ax2.plot([-phi_broad/2,-phi_broad/2],[0,1.1],color='k',
             linestyle='dashed',linewidth = 2)
    ax2.plot([ phi_broad/2, phi_broad/2],[0,1.1],color='k',
             linestyle='dashed',linewidth = 2)

    #ax2.set_xticks(np.arange(-200,300,50))
    ax2.set_xticks(np.arange(-50,55,5))
    ax2.set_xlim(-50,50)
    ax2.tick_params(labelsize=fs,axis='both')
    #ax2.set_xlim(-phi_max,phi_max)
    ax2.set_ylim(0,1.1)
    ax2.set_xlabel('Faraday Depth (rad m$^{-2}$)',fontsize=fs)
    ax2.set_ylabel('RMSF',fontsize=fs)
    ax2.grid()

    plt.savefig(file_out)
    
    #plt.suptitle(survey['name'],fontsize=fs)

    return

In [ ]:
gmims_hbn  = {'file':'gmims_hbn', 'name':'GMIMS-HBN', 
               'frange':[[1300e6,1750e6]],  
               'df':1e6}

gmims_full  = {'file':'gmims_full', 'name':'GMIMS', 
               'frange':[[300e6,1800e6]],  
               'df':0.5e6}

gmims_fullN  = {'file':'gmims_fullN', 'name':'GMIMS', 
               'frange':[[350e6,1800e6]],  
               'df':0.5e6}

gmims_full_rfi  = {'file':'gmims_full_rfi', 'name':'GMIMS', 
                   'frange':[[300e6,480e6],[724e6,750e6],[790e6,860e6],[900e6,935e6],
                             [970e6,1510e6],[1620e6,1800e6]],  
                   'df':0.5e6}

gmims_fullN_rfi  = {'file':'gmims_fullN_rfi', 'name':'GMIMS', 
                   'frange':[[350e6,360e6],[380e6,530e6],[580e6,620e6],[640e6,720e6],[780e6,860e6],
                             [900e6,1520e6],[1620e6,1800e6]],  
                   'df':0.5e6}

pegasus  = {'file':'gmims_full', 'name':'GMIMS', 
            'frange':[[700e6,1440e6]],  
            'df':0.5e6}

pegasus_rfi  = {'file':'gmims_full_rfi', 'name':'GMIMS', 
                'frange':[[724e6,750e6],[790e6,860e6],[900e6,935e6],[970e6,1440e6]],  
                'df':0.5e6}

dva2_emss  = {'file':'dva2_emss', 'name':'GMIMS MBN', 
              'frange':[[900e6,1750e6]],  
              'df':0.5e6}

chime_rfi  = {'file':'chime_rfi', 'name':'full CHIME RFI', 
              'frange':[[400e6,446e6],[460e6,483e6],
                        [506e6,514e6],[524e6,528e6],[537e6,542e6],[556e6,565e6],
                        [586e6,692e6],[698e6,729e6]],  
              'df':0.390625e6}

chime_full  = {'file':'chime_full', 'name':'full CHIME', 
              'frange':[[400e6,800e6]],  
              'df':0.390625e6}

chime_rfi_low  = {'file':'chime_rfi', 'name':'full CHIME RFI', 
              'frange':[[404e6,415e6],[428e6,440e6],[452e6,454e6],[460e6,483e6]],  
              'df':0.4e6}

dragons_high  = {'file':'dragons_high', 'name':'full DRAGONS 800-1030 MHz', 
              'frange':[[800e6,1030e6]],  
              'df':1e6}


#chime_rfi_high  = {'file':'chime_rfi', 'name':'full CHIME RFI', 
#              'frange':[[404e6,415e6],[428e6,440e6],[452e6,454e6],[460e6,483e6],
#                        [506e6,514e6],[524e6,528e6],[537e6,541e6],[556e6,565e6],
#                        [586e6,692e6],[698e6,729e6]],  
#              'df':0.4e6}

#dragons    = {'file':'dragons',   'name':'DRAGONS',   'fmin':350e6,  'fmax':1030e6, 'df':1e6}
#chime_full = {'file':'chime_full','name':'CHIME ',    'fmin':400e6,  'fmax':800e6,  'df':0.4e6}
#chime_low  = {'file':'chime_low', 'name':'CHIME low', 'fmin':400e6,  'fmax':440e6,  'df':0.4e6}
#chime_high = {'file':'chime_high','name':'CHIME high','fmin':600e6,  'fmax':800e6,  'df':0.4e6}
#gmims_hbn  = {'file':'gmims_hbn', 'name':'GMIMS HBN', 'fmin':1280e6, 'fmax':1750e6, 'df':1e6}
#gmims_lbs  = {'file':'gmims_lbs', 'name':'GMIMS LBS', 'fmin':300e6,  'fmax':480e6,  'df':0.5e6}
#staps      = {'file':, 'name':,'fmin': , 'fmax': ,  'df':}
#pegasus    = {'file':, 'name':, fmin': , 'fmax': ,  'df':}

In [ ]:
survey     = chime_full
phi_max    = 200 # range of FD values to plot for RMSF (+/-)
dphi       = 0.1 # FD sampling
color      = 'C0' # colors to use for plotting
combine    = False # Set True if want to combine frequency coverage from chosen surveys
file_out   = 'chime_full_RMSF.jpg' # name of the file to save

## Get the RMSF

In [ ]:
%%time
freq_ranges = survey['frange']

fcombine = []
for frange in freq_ranges:
    fcombine.extend(list(np.arange(frange[0],frange[1]+survey['df'],survey['df'])))

with open('combo_freq.txt', 'w') as filehandle:
    for i in range(0,len(fcombine)):
        filehandle.write(f"{fcombine[i]}\n")

cmd = 'python3 calculate_RMSF_AO.py combo_freq.txt -m '+str(phi_max)+' -d '+str(dphi)+' -o combo'
!{cmd}

########################################################
# Tadpole stuff
#freq_file = '/srv/data/chime/tadpole_cutout/IQU_400_729/good_freq_400_729.txt'
#cmd = 'python3 calculate_RMSF_AO.py '+freq_file+' -m '+str(phi_max)+' -d '+str(dphi)+' -o combo'
#!{cmd}
########################################################

phi, rmsf, phi_broad, res = get_combo_survey_params()

#cmd = 'rm *.txt'
#!{cmd}
print(phi_broad)
print(res)


In [ ]:
make_the_plot_rfi(survey,color,phi_broad,phi,rmsf,save_plot=True,file_out=file_out)

#make_the_plot(survey,colors,freqs_all,phi_broad_all,phi_all,rmsf_all,fs=24,
#              phi_combo=None, rmsf_combo=None, phi_broad_combo=None,inc_legend=False,
#              do_combo=combine,file_out=file_out)
